In [2]:
import json

def get_environment_variable(config_file, variable_name):
    """
        Getting the Storage Account connection string from the configuration file

        :param config_file: the json file with the environment variables
        :param variable_name: the variable name
    """
    try:
        # loading config JSON file
        with open("config.json", "r") as config_file:
            config = json.load(config_file)

        # picking variable
        var_name = config[variable_name]

        if var_name:
            print(f"Succcess getting {variable_name}")
            return var_name
        else:
            print("Failed to retrieve the {variable_name}.")
            return None
    except Exception as e:
        print(f"Caught an exception: {e}")
    return None



In [3]:
## Getting the connection string

storage_account_name = get_environment_variable("config_file","STORAGE_ACCOUNT_NAME")
resource_group_name =  get_environment_variable("config_file","RESOURCE_GROUP_NAME")
result = !az storage account show-connection-string --name {storage_account_name} --resource-group {resource_group_name}

Succcess getting STORAGE_ACCOUNT_NAME
Succcess getting RESOURCE_GROUP_NAME


In [33]:
import json
## Copying the string in the file
if result:
    json_str = "\n".join(result)
    connection_string_data = json.loads(json_str)  # Json to dict
    
    # Salvar a conexão no arquivo config.json
    with open("config.json", "r+") as file:
        config = json.load(file)  # Carregar o JSON existente
        config["CONNECTION_STRING"] = connection_string_data["connectionString"]  # Updating variable
        file.seek(0)  #go to file begining
        json.dump(config, file, indent=4)  # saving file
        file.truncate()  

    print("Success getting and writting the string!")
else:
    print("Error")

Success getting and writting the string!


In [3]:
!pip install azure-storage-blob


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.4/408.4 kB 4.4 MB/s eta 0:00:000:00:010:00:01


In [ ]:
from azure.storage.blob import BlobServiceClient

def connect_to_blob_storage(connection_string):
    """
      Connect to Azure Blob Storage using a connection string and return the blob service client.
    
    :param connection_string: The Azure Storage account connection string.
    """
    try:
        # Create the BlobServiceClient object
        blob_service_client = BlobServiceClient.from_connection_string(connection_string)
        return blob_service_client 
    except Exception as e:
        print(f"Error connecting to Blob Storage: {e}")

In [38]:
def create_containers(blob_service_client,container_name):
    # Create the container
    try:
        container_client = blob_service_client.create_container(container_name)
        print(f"Container '{container_name}' created successfully!")
    except Exception as e:
        print(f"Error creating container: {e}")

In [5]:
def list_containers(blob_service_client):
    """
        List all containers in one storage account

        :param blob_service_client: the connect blob storage service for the Azure Account
    """
    # List the containers in the storage account
    containers = blob_service_client.list_containers()
    print("Containers in the storage account:")
    for container in containers:
        print(f" - {container['name']}")


In [6]:
from datetime import datetime

def upload_file_blob(blob_service_client, file_name, container_name, file_path):
    """
        Upload a file to Azure Blob Storage.
        
        :param blob_service_client: the connected blob service client
        :param blob_name: the name of the file after upload
        :param container_name: the name of the container that shoud receive the file
        :param file_path: the full path of the file 
    """
    try:
        # Get the BlobClient
        blob_client = blob_service_client.get_blob_client(container=container_name, blob=file_name)
        
        # Upload the file
        with open(file_path, "rb") as file_data:
            blob_client.upload_blob(file_data, overwrite=True) # with  overwrite=True the file can be overwrited
        current_time = datetime.now().strftime("%d/%m/%Y %H:%M")
        print(f"File '{file_path}' uploaded to container '{container_name}' as blob '{file_name} at {current_time} '.")
    except Exception as e:
        print(f"Error uploading blob: {e}")

## Getting env variables

In [34]:
# 1. Get connection string of the Storage Account
config_file_path = "config.json"  
connection_string = get_environment_variable(config_file_path,"CONNECTION_STRING")

Succcess getting CONNECTION_STRING


In [35]:
#2.Connecting the Blob service Storage for the Storage Account
blob_service_client = connect_to_blob_storage(connection_string)
if blob_service_client:
    print(f"The blob service Client has been created")
else:
    print(f"Unable to connect")

The blob service Client has been created


In [36]:
#3.Listing the containers in the Storage Account Blob Service
list_containers(blob_service_client)

Containers in the storage account:


In [39]:
# 4.Upload one image in the blob storarage
file_name ="imaged.png"
container_name ="bronze"
file_path ="/home/itaira/Python/projects/Azure/imagedp203.png"

upload_file_blob(blob_service_client,file_name,container_name,file_path )


File '/home/itaira/Python/projects/Azure/imagedp203.png' uploaded to container 'bronze' as blob 'imaged.png at 02/12/2024 16:10 '.
